In [1]:
import pandas as pd

In [6]:
with open("./订单表.csv") as f:
    df_order_list = pd.read_csv(f)
df_order_list.head()

,订单日期,年份,订单数量,产品ID,产品型号名称,产品名称,产品类别,单价,销售金额,产品成本,利润,客户ID,交易类型,销售区域ID,销售大区,国家,区域
0,2016/1/1,2016,1,528,Mountain Tire Tube,车胎和内胎,配件,4.99,4.99,1.8663,3.1237,12411,1,4,西南区,中国,大中华区
1,2016/1/2,2016,1,528,Mountain Tire Tube,车胎和内胎,配件,4.99,4.99,1.8663,3.1237,16720,1,4,西南区,中国,大中华区
2,2016/1/2,2016,1,528,Mountain Tire Tube,车胎和内胎,配件,4.99,4.99,1.8663,3.1237,25967,1,4,西南区,中国,大中华区
3,2016/1/5,2016,1,528,Mountain Tire Tube,车胎和内胎,配件,4.99,4.99,1.8663,3.1237,23689,1,4,西南区,中国,大中华区
4,2016/1/6,2016,1,528,Mountain Tire Tube,车胎和内胎,配件,4.99,4.99,1.8663,3.1237,12978,1,4,西南区,中国,大中华区


In [8]:
with open("./客户.csv") as f:
    df_customer_list = pd.read_csv(f)
df_customer_list.head()

,客户ID
0,28389
1,21768
2,25863
3,14501
4,11003


In [34]:
with open("./产品表.csv") as f:
    df_object_list = pd.read_csv(f)
df_object_list.head()

,产品类别,产品ID,产品型号,产品名称
0,配件,528,Mountain Tire Tube,车胎和内胎
1,配件,480,Patch kit,车胎和内胎
2,配件,537,HL Mountain Tire,车胎和内胎
3,配件,529,Road Tire Tube,车胎和内胎
4,配件,536,ML Mountain Tire,车胎和内胎


In [9]:
with open("./日期表.csv") as f:
    df_date_list = pd.read_csv(f)
df_date_list.head()

,日期,年度,季度,月份,日,年度季度,年度月份,星期几
0,2013/7/1,2013,Q3,7,1,2013Q3,201307,1
1,2013/7/2,2013,Q3,7,2,2013Q3,201307,2
2,2013/7/3,2013,Q3,7,3,2013Q3,201307,3
3,2013/7/4,2013,Q3,7,4,2013Q3,201307,4
4,2013/7/5,2013,Q3,7,5,2013Q3,201307,5


In [23]:
# 提取每一天每一位顾客购买的产品列表
transaction_list = []
df_order_group_by_date = df_order_list.groupby("订单日期")
for date, df_daily_order in df_order_group_by_date:
    df_daily_order_group_by_customer = df_daily_order.groupby("客户ID")
    for customer_id, df_customer_daily_order in df_daily_order_group_by_customer:
        transaction_list.append([str(ele) for ele in list(df_customer_daily_order["产品ID"])])

df_transaction_list = pd.DataFrame(transaction_list)
df_transaction_list.head()

,0,1,2,3,4,5,6,7
0,344,None,None,None,None,None,None,None
1,314,None,None,None,None,None,None,None
2,328,None,None,None,None,None,None,None
3,311,None,None,None,None,None,None,None
4,314,None,None,None,None,None,None,None


In [25]:
# 使用空字节填充NaN
dataset = df_transaction_list.fillna('')
dataset.head()

,0,1,2,3,4,5,6,7
0,344,,,,,,,
1,314,,,,,,,
2,328,,,,,,,
3,311,,,,,,,
4,314,,,,,,,


In [27]:
# 将每一行数据组合成一列，用逗号分隔
dataset_combined = dataset[0]
for i in range(1, len(dataset.columns)):
    dataset_combined = dataset_combined + ',' + dataset[i]
dataset_combined = pd.DataFrame(dataset_combined)
dataset_combined.columns = ['transactions']
dataset_combined.head()

,transactions
0,"344,,,,,,,"
1,"314,,,,,,,"
2,"328,,,,,,,"
3,"311,,,,,,,"
4,"314,,,,,,,"


In [28]:
# 将物品转化为one hot编码
dataset = dataset_combined['transactions'].str.get_dummies(',')
dataset.head(5)

,214,217,222,225,228,231,234,237,310,311,...,594,595,596,597,598,599,600,604,605,606
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
# 使用mlxtend提炼数据中的关联规则
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

frequent_itemsets = apriori(dataset, min_support=0.015, use_colnames=True)
frequent_itemsets = frequent_itemsets.sort_values(by="support" , ascending=False)
frequent_itemsets = frequent_itemsets.reset_index(drop=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.5)
rules = rules.sort_values(by="lift" , ascending=False)
rules = rules.reset_index(drop=True)
print("频繁项集：", frequent_itemsets)
print("关联规则：", rules)

频繁项集：      support    itemsets
0   0.153632       (477)
1   0.115541       (480)
2   0.112065       (528)
3   0.085958       (529)
4   0.080744       (214)
5   0.079296       (225)
6   0.076906       (222)
7   0.076798       (485)
8   0.075494       (217)
9   0.073322       (478)
10  0.061989       (479)
11  0.061264  (477, 478)
12  0.055073  (477, 479)
13  0.053842       (530)
14  0.050547       (537)
15  0.042038       (536)
16  0.037801       (538)
17  0.034651  (528, 537)
18  0.033855       (541)
19  0.033529       (539)
20  0.032877       (484)
21  0.031212       (535)
22  0.030994       (540)
23  0.029256  (541, 530)
24  0.028605  (528, 480)
25  0.028098  (528, 536)
26  0.026541       (487)
27  0.022666  (477, 225)
28  0.021942  (529, 539)
29  0.021580  (529, 480)
30  0.021544  (540, 529)
31  0.020132  (538, 529)
32  0.018068       (465)
33  0.017670       (463)
34  0.017308  (528, 535)
35  0.016402  (528, 217)
36  0.016366       (234)
37  0.016040       (467)
38  0.016004       

In [56]:
# 打印前三个最畅销的商品组合
for i, items in enumerate(frequent_itemsets['itemsets'][:3]):
    print('No.' + str(i + 1) + ' sold itemset')
    for item in items:
        print(df_object_list[df_object_list["产品ID"] == int(item)])
    print()

No.1 sold itemset
    产品类别  产品ID          产品型号    产品名称
156   配件   477  Water Bottle  水壶和水壶架

No.2 sold itemset
  产品类别  产品ID       产品型号   产品名称
1   配件   480  Patch kit  车胎和内胎

No.3 sold itemset
  产品类别  产品ID                产品型号   产品名称
0   配件   528  Mountain Tire Tube  车胎和内胎



In [57]:
# 打印lift前三的商品组合
i = 1
for antecedents, consequents in zip(rules['antecedents'][:3], rules['consequents'][:3]):
    print('No.' + str(i) + ' support item group')
    print('antecedents:')
    for item in antecedents:
        print(df_object_list[df_object_list["产品ID"] == int(item)])
    print('consequents:')
    for item in consequents:
        print(df_object_list[df_object_list["产品ID"] == int(item)])
    i += 1
    print()

No.1 support item group
antecedents:
  产品类别  产品ID          产品型号   产品名称
8   配件   541  Touring Tire  车胎和内胎
consequents:
  产品类别  产品ID               产品型号   产品名称
5   配件   530  Touring Tire Tube  车胎和内胎

No.2 support item group
antecedents:
  产品类别  产品ID               产品型号   产品名称
5   配件   530  Touring Tire Tube  车胎和内胎
consequents:
  产品类别  产品ID          产品型号   产品名称
8   配件   541  Touring Tire  车胎和内胎

No.3 support item group
antecedents:
  产品类别  产品ID            产品型号   产品名称
3   配件   529  Road Tire Tube  车胎和内胎
consequents:
  产品类别  产品ID          产品型号   产品名称
6   配件   540  HL Road Tire  车胎和内胎

